# Neural Networks: BatchNorm + ReLU

In the previous tutorial, we constructed our first neural network composed of linear layers and sigmoid activations. In this notebook, we will try to further improve the network by replacing the sigmoid with a ReLU activation and by adding batch normalization layers.

### Activation Functions

#### Disadvantrages of the sigmoid activation
So far, we have only used the sigmoid activation to enable non-linear mappings. However, in state-of-the-art (SOTA) networks, sigmoid activation can hardly be found as better activation functions have been identified.

Looking at the graph below, we can see the sigmoid activation function (left) and the derivative of the function (right). As can be seen, the maximum of the derivative is 0.25, and the derivative becomes almost zero for large $x$  (i.e., > 5) or small values (i.e., < -5). 

<img src="imgs/sigmoid.png" width="400px" />

Since backpropagation utilizes on the derivative of the function for updating the weights, hardly an change will happen for large/small $x$ values. One we enter this area, the gradient vanishes and training gets stuck. <br/>
Furthermore, considering that the maximum derivative is 0.25, having multiple layers with sigmoid activation, will cause the gradient to become smaller. For example, after two layers the maximum would be ($\frac{1}{4} \times \frac{1}{4} = \frac{1}{16}$) and so on. This hinders trying deeper networks. We refer to this phenomenon as **vaninishing gradient** problem.

#### Other activation functions

Nowadays, we mostly find ReLU or LeakyReLU being as they can cope with the vanishing gradient problem. In the graph below shows the ReLU activation that mathematically can be defined as $f(x) = \text{max}(0,x)$. Consequently, the derivative of the function is 1 for values larger than zero. This prevents the gradient from vanishing if ReLU activations are applied repeatedly. 

<img src="imgs/relu.png" width="300px" />

However, there still is another problem. Once we $x$ becomes less than 0, the gradient becomes 0 and we are able to recover from this state due to a lack of gradient information. We say that the activation got **stuck**. A solution to this can be to prevent constant outputs, in the case of the ReLU the solution is a leaky ReLU which has a slight slope in the negative region.

<img src="imgs/leaky_relu.png" width="300px" />


### Batch Normalization

#### What is batch normalization and why do we use it?

In the previous example, we already standardized the *input features* which ensures that the input distribution to have zero mean and unit standard deviation. <br/>
Assuming that a linear layer is initialized with random weights of around 0, this, this ensures the linear layer's output is in the area of typical activation functions. This improves the adaptation rate as we avoid low/zero gradients. 

However, once networks get deeper, the chance increases that some linear layer in the network does map to a "different area", implying that the distribution at a particular layer isn't a *N(0,1)* distribution. This phenomenon is referred to as **internal covariate shift (ICS)**. In other words, the term "internal covariance shift" refers to a change of the output distribution due to a shift in the layer parameters. Since layers of a neural network are dependent on each other, the following layer depends on the output distribution of the previous layer. This has an impact on learning.

The idea of **Batch normalization** is simply to reduce this covariate shift by normalizing the output of intermediate layers. This has an impact on learning.

Batch normalization *usually* has the following benefits:
- Allows for higher learning rates as it prevents fast deterioration to low/zero
gradients
- Makes sigmoid-type activation functions more usable (for the same reason)
- Has regularization properties which can sometimes have similar effects as dropout.

#### How to apply batch normalization

As mentioned previously, we would like to standardize the output of an intermediate layer in the network. Consequently, for a layer with $d$-dimensional input $x = (x^{(1)}, ..., x^{(n)})$, we will normalize each dimension such that

$$\hat{x^{(k)}} = \frac{x^{(k)} - \text{E}[x]}{{\text{Var}[x]}}$$

where the expectation and variance are computer over the training dataset. 

However, note that **simply normalizing each input of a layer may change what layer can represent**. For instance, let's assume that we have a linear layer followed by batch normalization, and we want it to learn the identity mapping. It turns out that due batch normalization, such mapping can only be learned if the expectation is 0 and variance is 1. 

However, to work around this issue, we perform an additional transformation directly after the normalization step. For the activation $\hat{x^{(k)}}$, a learned pair of parameters $\lambda^{(k)}$ and $\beta^{(k)}$, which scales and shifts the normalized value.

$$y^{(k)} = \lambda^{(k)} \hat{x^{(k)}} + \beta^{(k)}$$

These parameters are learned along with the original model, and restore the presentative power of the network.

#### Batch normalization with mini-batch statistics

Unfortunately, in practice, computing the expection and variance across the entire dataset is impractical. Therefore, a simplification is made: Since we use mini-batches in stochastic gradient training, each mini-batch produces estimates of the mean and variance of each activation. We can use this statistics to estimate the expectation and variance for the entire dataset.

**During training**, batch normalization for a given batch $B = \{x_1, ..., x_m \}$ where the parameters $\lambda$ and $\beta$ are learned is implemented as follows:

$$\mu_B = \frac{1}{m} \sum_{i=1}^m x_i$$

$$\sigma^2_B = \frac{1}{m} \sum_{i=1}^m (x_i-\mu_B)^2$$

$$\hat{x_i} = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} $$

$$y^{(k)} = \lambda \hat{x^{(k)}} + \beta$$

**During inference**, the learned parameters $\lambda$ and $\beta$ are used to normalize the distribution.

### Basic imports

In [4]:
from sklearn.datasets import fetch_california_housing
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import trange
import matplotlib.pyplot as plt
import random

In [2]:
# To ensure reproducability
np.random.seed(42)

In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Fetch the california housing dataset

In [ ]:
dataset = fetch_california_housing(as_frame=True)

In [ ]:
housing_df = dataset['data']
target_df = dataset['target']

In [ ]:
# Insert the housing prices in the housing df
housing_df['HousePrice'] = target_df

## Prepare the training and testing set


### Splitting the dataframe (using sklearn)

In [ ]:
train_df, test_df = train_test_split(housing_df, test_size=0.2)

In [ ]:
# Briefly check whether we have the correct set sizes
print('Test ratio: ', len(test_df) / (len(train_df) + len(test_df)))

In [ ]:
feature_columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
target_column = 'HousePrice'

x_train = train_df[feature_columns].values
y_train = train_df[['HousePrice']].values

x_test = test_df[feature_columns].values
y_test = test_df[['HousePrice']].values

### Extending the network

We will now try to improve the network architecture that was introduced in the previous tutorial. <br/>
More precisely, we will make the following adjustments to our network:

- Replace the sigmoid with ReLU (or LeakyReLU) activations
- Add BatchNormalization layers
- Increase the number of parameters / layers

The final structure of our network should look as follows:

<img src="imgs/linear_model_2.png" width="1000px"/>

In [5]:
import torch.nn as nn
import torch

In [ ]:
input_dims = len(feature_columns)
output_dims = 1

In [ ]:
model = nn.Sequential(
    nn.Linear(input_dims, 512, bias=False),
    nn.BatchNorm1d(512),
    nn.LeakyReLU(),
    nn.Linear(512, 256, bias=False),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(),
    nn.Linear(256, 256, bias=False),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(),
    nn.Linear(256, 128, bias=False),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 1),
)

We can now print the network to verify the network architecture is correct ...

In [ ]:
print(model)

In [ ]:

x = torch.rand((64, 8))
pred = model(x)
print(pred.shape)

### Preprocess the samples 

Before training the neural network, input features should be normalized (very often to the range -1 to 1) or standardized, and converted to a PyTorch tensor.

#### Standardize the input features

In this example, we will use Scikit's StandardScaler to standardized the input features. In future examples, we will start to use PyTorch's "on-board capabilities" to preprecess the input features (e.g., https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#### Convert all samples to a PyTorch tensor

The entire dataset (`x_train` and `x_test`) is stored as numpy array at this point and we still have convert it to a PyTorch tensor.

Tensors are similar to numpy ndarrays, except that tensors can run on GPUs or other hardware accelerators. In fact, tensors and NumPy arrays can often share the same underlying memory, eliminating the need to copy data (see Bridge with NumPy). Tensors are also optimized for automatic differentiation.

In [ ]:
import torch

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
print(x_train.shape)
print(type(x_train))

### Train the neural network

In [ ]:
# Fix seed to ensure reproducibility
np.random.seed(42)
torch.manual_seed(0)

#### Helper function that computes for RMSE for a given dataset

The *evaluate_model_performance()* function receives a set of features and their target levels as its input, and evaluates the model performance using the root mean square error. In order to do this, the network must be switched to *inference mode* (`model.eval()`).

In [ ]:
def evaluate_model_performance(x, y, batch_size=128):
    
    assert x.shape[0] == y.shape[0], 'Feature and target labels a different number of samples'
    
    # During training, the batch norm layer keeps a running estimate of its computed mean and variance. 
    # At inference time, the mean/variance must not be updated. The estimed mean/variance is used for normalization.
    model.eval()
    
    total_error = 0.
    
    num_samples = x.shape[0]
    
    # No gradient have to be computed at inference time. We can disable gradient computation by means of the
    # no_grad() context manager. This will reduce the memory consumption.
    with torch.no_grad():
    
        for indices in batch(range(num_samples), batch_size):
            
            # Convert the sample to a Pytorch tensor
            sample_x = x[indices]
            sample_y = y[indices]
            
            pred = model(sample_x)
            
            total_error += torch.sum((pred - sample_y) **2)
            
    rmse = torch.sqrt(total_error / num_samples) * 100000
    
    # Switch the network back to training mode since want to continue training.
    model.train()
 
    return rmse.numpy()


#### Start training

The training procedure for a neural network looks as follows: First, sample a random set of samples from the training set. This randomly drawn subset is referred to as a **batch**.

The training batch is fed through the network to obtain the prediction (output of the network). Next, we need to update the parameters (weights) of the network. To accomplish this, we first have to compute the loss (a.k.a. cost function) and then compute the gradient with respect to every parameter in the network. Once we obtain the individual gradients, we can update the parameters. For updating the parameters, we use an optimizer such as ADAM or SGD.

In [ ]:
from torch.optim import Adam
import torch.nn.functional as F

In [ ]:
batch_size = 128
num_epochs = 300

In [ ]:
# We will use the ADAM optimizer to find the "optimal" network weights.
# The ADAM optimization is a stochastic gradient descent method that is based on 
# adaptive estimation of first-order and second-order moments.
optimizer = Adam(model.parameters(), lr=1e-6)

In [ ]:
train_rmse = []

num_samples = x_train.shape[0]

for epoch in trange(num_epochs):
    
    # Samples should be drawn in random order.
    # Hence, we create a list of indices (in random order) from which we can sample from.
    shuffled_indices = list(range(num_samples))
    random.shuffle(shuffled_indices)
    
    batch_iter = batch(shuffled_indices, 128)
   
    for batch_idx, sample_indices in enumerate(batch_iter):
  
        # Clear all gradients
        optimizer.zero_grad()

        batch_x = x_train[sample_indices]
        batch_y = y_train[sample_indices]

        # Push the training batch through the neural network
        pred = model(batch_x)

        # Compute the mean squared error --> Our goal is to minimize this error
        loss = F.mse_loss(pred, batch_y)

        # Trigger backpropagation. This computes all gradients that have to be optimized.
        loss.backward()

        # Update the weights based on the obtained gradient values
        optimizer.step()

        if batch_idx % 50 == 0:
            train_rmse.append(evaluate_model_performance(x_train, y_train))
            

In [ ]:
plt.plot(list(range(len(train_rmse))), train_rmse)
plt.title('Train RMSE')

### Performance evaluation

In [ ]:
train_rmse = evaluate_model_performance(x_train, y_train)
test_rmse = evaluate_model_performance(x_test, y_test)

In [ ]:
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)